In [73]:
import os
import sys
import pandas as pd
import logging
from datetime import datetime, date
from google.cloud import bigquery
from hashlib import md5

# emission data file name
filename = './data/Emissions_by_Country_2002-2022.csv'
PROJECT_NAME = "emissions-team-project"
DATASET_NAME = "emissions"

# **** TABLE SCHEMAS ****

TABLE_METADATA = {
    'fct_emissions': {
        'table_name': 'fct_emissions',
        'schema': [
            # indexes are written if only named in the schema
            bigquery.SchemaField('country_code_year', 'string', mode='REQUIRED', max_length=128),
            bigquery.SchemaField('country_code', 'string', mode='REQUIRED'),
            bigquery.SchemaField('year', 'date', mode='REQUIRED'),
            bigquery.SchemaField('total_em', 'float64', mode='REQUIRED'),
            bigquery.SchemaField('coal_em', 'float64', mode='REQUIRED'),
            bigquery.SchemaField('oil_em', 'float64', mode='REQUIRED'),
            bigquery.SchemaField('gas_em', 'float64', mode='REQUIRED'),
            bigquery.SchemaField('cement_em', 'float64', mode='REQUIRED'),
            bigquery.SchemaField('flaring_em', 'float64', mode='REQUIRED'),
            bigquery.SchemaField('other_em', 'float64', mode='REQUIRED'),
            bigquery.SchemaField('created_at', 'float64', mode='REQUIRED'),
            bigquery.SchemaField('modified_at', 'float64', mode='REQUIRED'),
        ],
    }
}

# **** SETUP LOGGING ****
# setup logging and logger
logging.basicConfig(            # setting up the root logger
    format='[%(levelname)-5s][%(asctime)s][%(module)s:%(lineno)04d] : %(message)s',
    level=logging.INFO,
    stream=sys.stdout
)
logger: logging.Logger = logging.getLogger('root')      # alias the root logger as `logger`
logger.setLevel(logging.DEBUG)                          # programmatically reassign the logging level


# **** BIGQUERY CLIENT ****
logger.debug(f"Creating bigquery client")
client = bigquery.Client()

logger.info(f"Setup Completed")

[DEBUG][2023-01-18 09:27:02,075][3564589987:0049] : Creating bigquery client
[INFO ][2023-01-18 09:27:02,083][3564589987:0052] : Setup Completed


## Loading Emission_by_Country CSV File

In [79]:
# *** always perform checks first ***
# check if the file exists
assert os.path.exists(filename), f"Data file does not exists: '{filename}'"
# check if the file contains any data. Header row alone is 78 bytes. size > 78
assert os.path.getsize(filename) > 78, f"Data file size incorrect; does not seem to contain data: '{filename}'"

# load into dataframe
df = pd.read_csv(
    filename, 
    header=0,
    parse_dates=['Year'], 
    infer_datetime_format=True, 
    on_bad_lines='warn',
    )
logger.info(f"loaded {len(df.index)} rows from {filename}")

# *** always perform check at the end ***
# check schema: contains all expected columns?
expected_columns = ['Country', 'ISO 3166-1 alpha-3', 'Year', 'Total', 'Coal', 'Oil', 'Gas', 'Cement', 'Flaring', 'Other', 'Per Capita']
for col in expected_columns:
    assert col in list(df.columns), f"Data file missing required column: {col}"

#convert columns name to all lower case
df= df.rename(columns=str.lower)
df = df.rename(columns={'iso 3166-1 alpha-3': 'country_code','total': 'total_em', 'coal': 'coal_em', 'oil': 'oil_em', 'gas': 'gas_em', 'cement': 'cement_em', 'flaring': 'flaring_em', 'other': 'other_em'})

# log data column data types
logger.debug(df.dtypes)
display(df.head(5))

[INFO ][2023-01-18 09:31:03,904][2264186883:0015] : loaded 63104 rows from ./data/Emissions_by_Country_2002-2022.csv
[DEBUG][2023-01-18 09:31:03,911][2264186883:0028] : country                 object
country_code            object
year            datetime64[ns]
total_em               float64
coal_em                float64
oil_em                 float64
gas_em                 float64
cement_em              float64
flaring_em             float64
other_em               float64
per capita             float64
dtype: object


,country,country_code,year,total_em,coal_em,oil_em,gas_em,cement_em,flaring_em,other_em,per capita
0,Afghanistan,AFG,1750-01-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,1751-01-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,1752-01-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,1753-01-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,1754-01-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
#add created_at and modified_at columns
df['created_at'] = pd.Timestamp('today').strftime("%Y-%m-%d")
df['modified_at'] = 'None'

display(df)

,country,country_code,year,total_em,coal_em,oil_em,gas_em,cement_em,flaring_em,other_em,per capita,created_at,modified_at
0,Afghanistan,AFG,1750-01-01,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-18,None
1,Afghanistan,AFG,1751-01-01,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-18,None
2,Afghanistan,AFG,1752-01-01,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-18,None
3,Afghanistan,AFG,1753-01-01,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-18,None
4,Afghanistan,AFG,1754-01-01,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-18,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63099,Global,WLD,2017-01-01,36096.739276,14506.973805,12242.627935,7144.928128,1507.923185,391.992176,302.294047,4.749682,2023-01-18,None
63100,Global,WLD,2018-01-01,36826.506600,14746.830688,12266.016285,7529.846784,1569.218392,412.115746,302.478706,4.792753,2023-01-18,None
63101,Global,WLD,2019-01-01,37082.558969,14725.978025,12345.653374,7647.528220,1617.506786,439.253991,306.638573,4.775633,2023-01-18,None
63102,Global,WLD,2020-01-01,35264.085734,14174.564010,11191.808551,7556.290283,1637.537532,407.583673,296.301685,4.497423,2023-01-18,None


In [81]:
from hashlib import md5

cols = ['country_code', 'year']
# first define a generic function that returns the md4 hash for
# any combination of values
def get_hash(*cols) -> str:
    """return the md5 hash of all parameters"""
    value = '-'.join([str(x) for x in cols])
    return md5(value.encode(encoding='utf-16')).hexdigest()

logger.info(f"assigning country_code_year: using md5 hash of country_code and year")

# customer_id = md5 hash of customer first and last name
df['country_code_year'] = df.apply(lambda row: get_hash(row.country_code, row.year), axis=1)
# set index by customer_id
df = df.set_index(keys='country_code_year')

logger.info(f"country_code_year generated")
display(df)

[INFO ][2023-01-18 09:31:22,934][1071718710:0011] : assigning country_code_year: using md5 hash of country_code and year
[INFO ][2023-01-18 09:31:26,935][1071718710:0018] : country_code_year generated


,country,country_code,year,total_em,coal_em,oil_em,gas_em,cement_em,flaring_em,other_em,per capita,created_at,modified_at
country_code_year,,,,,,,,,,,,,
64038db0ba9ee130cfe7ba09a02d99d2,Afghanistan,AFG,1750-01-01,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-18,None
0b6d630191f36d9545be317a8a7b0a75,Afghanistan,AFG,1751-01-01,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-18,None
b9287500248d689e9d6fca351cd5fde2,Afghanistan,AFG,1752-01-01,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-18,None
c4d61bf7975941be6c20b9ea5fc87025,Afghanistan,AFG,1753-01-01,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-18,None
07c9a388306297ff3b7862e0be54c437,Afghanistan,AFG,1754-01-01,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-18,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59495ce6483e0a2b64ce8cf5bcad6f36,Global,WLD,2017-01-01,36096.739276,14506.973805,12242.627935,7144.928128,1507.923185,391.992176,302.294047,4.749682,2023-01-18,None
bd367da414c7a24180f8bd4e8ba4e7ec,Global,WLD,2018-01-01,36826.506600,14746.830688,12266.016285,7529.846784,1569.218392,412.115746,302.478706,4.792753,2023-01-18,None
1d4f24b12858fbb44ed6c372a36273a2,Global,WLD,2019-01-01,37082.558969,14725.978025,12345.653374,7647.528220,1617.506786,439.253991,306.638573,4.775633,2023-01-18,None


In [72]:
# *** generic load function ***
def load_table(
    df: pd.DataFrame, 
    client: bigquery.Client, 
    table_name: str, 
    schema: bigquery.SchemaField, 
    create_disposition: str = 'CREATE_IF_NEEDED', 
    write_disposition: str = 'WRITE_TRUNCATE'
    ) -> None:
    """load dataframe into bigquery table

    Args:
        df (pd.DataFrame): dataframe to load
        client (bigquery.Client): bigquery client
        table_name (str): full table name including project and dataset id
        schema (List[bigquery.SchemaField]): table schema with data types
        create_disposition (str, optional): create table disposition. Defaults to 'CREATE_IF_NEEDED'.
        write_disposition (str, optional): overwrite table disposition. Defaults to 'WRITE_TRUNCATE'.
    """
    # *** run some checks ***
    # test table name to be full table name including project and dataset name. It must contain to dots
    assert len(table_name.split('.')) == 3, f"Table name must be a full bigquery table name including project and dataset id: '{table_name}'"
    # setup bigquery load job:
    #  create table if needed, replace rows, define the table schema
    job_config = bigquery.LoadJobConfig(
        create_disposition=create_disposition,
        write_disposition=write_disposition,
        schema=schema
    )
    logger.info(f"loading table: '{table_name}'")
    job = client.load_table_from_dataframe(df, destination=table_name, job_config=job_config)
    job.result()        # wait for the job to finish
    # get the resulting table
    table = client.get_table(table_name)
    logger.info(f"loaded {table.num_rows} rows into {table.full_table_id}")


receipts = df[['receipt_id', 'receipt_num', 'receipt_date', 'customer_id', 'item_count', 'receipt_total']].drop_duplicates(keep='first')
logger.info(f"Preparing {len(receipts.index)} unique receipts to load to bigquery")
display(receipts.head(n=10))
# load to bigquery
table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{TABLE_METADATA['receipts']['table_name']}"
schema = TABLE_METADATA['receipts']['schema']
load_table(receipts, client, table_name, schema)

NameError: name 'List' is not defined